In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/11/29 18:58:54 INFO SparkContext: Running Spark version 2.4.3
21/11/29 18:58:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/29 18:58:54 INFO SparkContext: Submitted application: 765e9f69-5fd7-4eb9-84b5-c5ad0daf7236
21/11/29 18:58:54 INFO SecurityManager: Changing view acls to: oscar
21/11/29 18:58:54 INFO SecurityManager: Changing modify acls to: oscar
21/11/29 18:58:54 INFO SecurityManager: Changing view acls groups to: 
21/11/29 18:58:54 INFO SecurityManager: Changing modify acls groups to: 
21/11/29 18:58:54 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(oscar); groups with view permissions: Set(); users  with modify permissions: Set(oscar); groups with modify permissions: Set()
21/11/29 18:58:54 INFO Utils: Successfully started service 'sparkDrive

21/11/29 18:58:55 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4-sources.jar at spark://netrunner:40195/jars/mainargs_2.12-0.1.4-sources.jar with timestamp 1638208735211
21/11/29 18:58:55 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4.jar at spark://netrunner:40195/jars/mainargs_2.12-0.1.4.jar with timestamp 1638208735211
21/11/29 18:58:55 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1-sources.jar at spark://netrunner:40195/jars/os-lib_2.12-0.7.1-sources.jar with timestamp 1638208735211
21/11/29 18:58:55 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1.jar at spark://netrunner:40195/jars/os-lib_2.12-0.7.1.j

21/11/29 18:58:55 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/alexarchambault/argonaut-shapeless_6.2_2.12/1.2.0-M11/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar at spark://netrunner:40195/jars/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar with timestamp 1638208735216
21/11/29 18:58:55 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.19.v20190610/jetty-server-9.4.19.v20190610.jar at spark://netrunner:40195/jars/jetty-server-9.4.19.v20190610.jar with timestamp 1638208735216
21/11/29 18:58:55 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/io/argonaut/argonaut_2.12/6.2.3/argonaut_2.12-6.2.3.jar at spark://netrunner:40195/jars/argonaut_2.12-6.2.3.jar with timestamp 1638208735216
21/11/29 18:58:55 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.

Spark UI

import $ivy.$                                  

import $ivy.$                              


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@43085ba6
import spark.implicits._


defined function run

In [30]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [31]:
val races = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")
    .select("raceId", "year")

load at cmd30.sc:4

1 / 1

races: DataFrame = [raceId: string, year: string]

In [32]:
val driverSeasonWindow = Window.partitionBy("driverId", "year")

driverSeasonWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2f8baa6b

In [37]:
val results = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/results.csv")
    .where(col("driverId") === 830)
    .withColumn("position", col("position").cast(IntegerType))
    .join(races, "raceId")
    .where(col("year") === 2021)
    .withColumn("points", col("points").cast(IntegerType))
    .withColumn("averagePoints", round(avg(col("points")).over(driverSeasonWindow), 2))
    .withColumn("podium", when(col("position") === 1 || col("position") === 2 ||col("position") === 3, lit(1)).otherwise(lit(0)))
    .withColumn("podiumChance", round(sum(col("podium")).over(driverWindow) / count(col("podium")).over(driverWindow), 2))
    .withColumn("totalPodiums", sum(col("podium")).over(driverWindow))
    .filter(col("podium") === 1)
//     .select("raceId", "position", "podiumChance", "averagePoints", "totalPodiums")
    .show()

load at cmd36.sc:4

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

show at cmd36.sc:14

...

show at cmd36.sc:14

20 / 20

show at cmd36.sc:14

39 / 39

+------+--------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+----+-------------+------+------------+------------+
|raceId|resultId|driverId|constructorId|number|grid|position|positionText|positionOrder|points|laps|       time|milliseconds|fastestLap|rank|fastestLapTime|fastestLapSpeed|statusId|year|averagePoints|podium|podiumChance|totalPodiums|
+------+--------+--------+-------------+------+----+--------+------------+-------------+------+----+-----------+------------+----------+----+--------------+---------------+--------+----+-------------+------+------------+------------+
|  1052|   24967|     830|            9|    33|   1|       2|           2|            2|    18|  56|     +0.745|     5524642|        41|   2|      1:33.228|        208.984|       1|2021|        20.22|     1|        0.89|           8|
|  1053|   24986|     830|            9|    33|   3|       1|   